In [ ]:
import re
import locale
import pandas as pd
import requests
import os

from bs4 import BeautifulSoup

# pd.set_option('display.max_rows', 100)

# Configura o locale para o formato de número brasileiro
locale.setlocale(locale.LC_ALL, 'pt_BR.utf8')

In [ ]:
def obter_texto_visivel(url):
    try:
        # Fazer a solicitação HTTP para a URL
        response = requests.get(url)
        response.raise_for_status()  # Verificar se a solicitação foi bem-sucedida

        # Criar um objeto BeautifulSoup para analisar o conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding="utf-8")

        # Extrair o texto visível da página (excluindo o HTML)
        texto_visivel = soup.get_text()

        return texto_visivel
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter a página: {e}")
        return None

In [5]:
def textFromURL(url):
    try:
        # Fazer a solicitação HTTP para a URL
        response = requests.get(url)

        # Verificar se a solicitação foi bem-sucedida
        response.raise_for_status()  

        # Criar um objeto BeautifulSoup para analisar o conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding="utf-8")

        # Extrair o texto visível da página (excluindo o HTML)
        texto_visivel = soup.get_text()

        # separa as linhas, e limpa os espaços iniciais, e tabulações
        linhas = [linha.replace("\t"," ").strip() for linha in texto_visivel.split("\n")]

        # junta o texto
        return "\n".join(linhas)
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter a página: {e}")
        return None

In [11]:
url = 'https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/202307.htm'
texto = textFromURL(url)

custos = []
for resultado in re.finditer(r"Vereador\(a\): (( |\.|\w+)*)\n(.+?)(?=Vereador)", texto, re.DOTALL):
    nome_vereador = resultado.group(1)
    dados = texto[resultado.start():resultado.end()]
    if "VEREADOR AFASTADO" in dados:
        continue

    total = re.search(r"TOTAL DO MÊS\s+([\d.,]+)", dados)
    if total:
        valor = locale.atof(total.group(1))
        custos.append({nome_vereador: valor})
        print(valor, "\t", nome_vereador)
    else:
        print(nome_vereador, dados)

custos

[]

In [8]:
custos.sort(key=lambda item: item[list(item.keys())[0]],reverse=True)
custos

[{'BETO DO SOCIAL': 43914.78},
 {'EDIR SALES': 38224.62},
 {'ISAC FELIX': 34605.97},
 {'JOÃO JORGE': 33590.02},
 {'LUANA ALVES': 31189.15},
 {'RODOLFO DESPACHANTE': 30504.19},
 {'SENIVAL MOURA': 30319.38},
 {'CELSO GIANNAZI': 30186.65},
 {'ELISEU GABRIEL': 29474.06},
 {'JAIR TATTO': 29000.28},
 {'BOMBEIRO MAJOR PALUMBO': 28461.29},
 {'MARCELO MESSIAS': 28417.95},
 {'GEORGE HATO': 28082.52},
 {'JOÃO ANANIAS': 27901.31},
 {'ATILIO FRANCISCO': 26970.31},
 {'RICARDO TEIXEIRA': 26952.06},
 {'SANDRA SANTANA': 26922.13},
 {'SILVIA DA BANCADA FEMINISTA': 26121.34},
 {'ELI CORRÊA': 25431.65},
 {'CAMILO CRISTÓFARO': 24319.05},
 {'DR SIDNEY CRUZ': 24038.18},
 {'ALESSANDRO GUEDES': 23509.54},
 {'LUNA ZARATTINI': 23131.94},
 {'DANILO DO POSTO DE SAÚDE': 21683.25},
 {'JANAINA LIMA': 21591.97},
 {'GILSON BARRETO': 21423.84},
 {'RUTE COSTA': 21330.79},
 {'SANSÃO PEREIRA': 20648.57},
 {'ELAINE DO QUILOMBO PERIFÉRICO': 18972.26},
 {'ANDRÉ SANTOS': 18960.05},
 {'JUSSARA BASSO': 18948.38},
 {'MANOEL DEL R

In [9]:
ano = {}
for n in range(1, 7, 1):
    file = f"2023{n:02d}"
    fileTXT = f"dados-{file}.txt"
    if os.path.exists(fileTXT):
        with open(fileTXT, "r", encoding="utf-8") as file:
            texto = file.readlines()
            file.close()
        texto = "".join(texto)
    else:
        url = f'https://sisgvarmazenamento.blob.core.windows.net/prd/PublicacaoPortal/Arquivos/{file}.htm'
        texto = textFromURL(url)
        with open(fileTXT, "w+", encoding="utf-8") as file:
            file.write(texto)
            file.close()

    for resultado in re.finditer(r"Vereador\(a\): (( |\.|\w+)*)\n(.+?)(?=Vereador)", texto, re.DOTALL):
        nome_vereador = resultado.group(1)
        if nome_vereador not in ano:
            ano[nome_vereador] = {"afastado": False}

        dados = texto[resultado.start():resultado.end()]
        if "VEREADOR AFASTADO" in dados:
            ano[nome_vereador]["afastado"] = True
            continue

        total = re.search(r"TOTAL DO MÊS\s+([\d.,]+)", dados)
        if total:
            valor = locale.atof(total.group(1))

            ano[nome_vereador][f"mes{n}"] = valor
ano

{'ADILSON AMADEU': {'afastado': False,
  'mes1': 3778.6,
  'mes2': 12170.89,
  'mes3': 4104.51,
  'mes4': 8518.69,
  'mes5': 43140.23,
  'mes6': 18122.71},
 'ALESSANDRO GUEDES': {'afastado': False,
  'mes1': 29392.35,
  'mes2': 28701.01,
  'mes3': 27023.94,
  'mes4': 25395.69,
  'mes5': 27773.76,
  'mes6': 23509.54},
 'ANDRÉ SANTOS': {'afastado': False,
  'mes1': 28304.03,
  'mes2': 18842.63,
  'mes3': 29981.71,
  'mes4': 16092.49,
  'mes5': 18396.37,
  'mes6': 18960.05},
 'ARSELINO TATTO': {'afastado': False,
  'mes1': 20695.96,
  'mes2': 19881.33,
  'mes3': 21437.47,
  'mes4': 23286.64,
  'mes5': 20536.02,
  'mes6': 17351.03},
 'ATILIO FRANCISCO': {'afastado': False,
  'mes1': 22502.54,
  'mes2': 26849.15,
  'mes3': 29436.92,
  'mes4': 28475.71,
  'mes5': 28587.13,
  'mes6': 26970.31},
 'AURELIO NOMURA': {'afastado': False,
  'mes1': 19482.7,
  'mes2': 16107.95,
  'mes3': 12956.13,
  'mes4': 13551.73,
  'mes5': 13032.11,
  'mes6': 16246.52},
 'BOMBEIRO MAJOR PALUMBO': {'afastado': Fa

In [10]:
df = pd.DataFrame.from_dict(ano, orient='index')
df.reset_index(inplace=True)
df.rename(columns={'index': 'vereador'}, inplace=True)
df.fillna(0, inplace=True)
df['total'] = df.iloc[:, -5:].sum(axis=1)
df = df[df['afastado'] == False]
df = df.drop('afastado', axis=1)
df = df.sort_values(by='total', ascending=False)
df = df.reset_index(drop=True)
df

,vereador,mes1,mes2,mes3,mes4,mes5,mes6,total
0,RODOLFO DESPACHANTE,1367.62,27927.23,38313.87,46948.15,45960.09,30504.19,189653.53
1,ELISEU GABRIEL,15430.96,43983.94,36941.35,31150.41,29450.14,29474.06,170999.90
2,ISAC FELIX,17572.46,30136.76,41415.87,34379.90,29449.85,34605.97,169988.35
3,LUANA ALVES,25236.11,31984.40,36838.80,34414.89,31549.55,31189.15,165976.79
4,JOÃO JORGE,26744.85,28063.26,34526.11,34401.89,32510.75,33590.02,163092.03
5,EDIR SALES,32118.75,20437.70,29623.07,32988.37,39319.99,38224.62,160593.75
6,GEORGE HATO,29803.01,34162.71,30952.28,29952.54,34172.14,28082.52,157322.19
7,SENIVAL MOURA,30326.25,30206.70,30070.38,31930.11,30654.66,30319.38,153181.23
8,ELI CORRÊA,24266.41,25701.64,27078.57,25594.96,41898.04,25431.65,145704.86
9,SILVIA DA BANCADA FEMINISTA,27309.34,30954.30,32875.74,26761.74,27295.80,26121.34,144008.92
